In [56]:
import numpy as np
import random
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos

## Classificadores

## Regressão Logística
### Funções auxiliares

In [57]:
def sigmoide(Z):
    return 1 / (1 + np.exp((-1 * Z)))

In [58]:
def kfold(qtd_linhas, seed = 42, k = 10):
    #Cria uma lista com os indices do dataset
    indices_dataset = list(range(qtd_linhas))
    
    #Define o tamanho do dataset
    tamanho_subset = round(len(indices_dataset) / k)
    
    #Cria uma lista de subconjuto de indices
    random.Random(42).shuffle(indices_dataset)
    lista_subsets = [indices_dataset[x : x + tamanho_subset] for x in range(0, len(indices_dataset), tamanho_subset)]

    kfolds = []
    for i in range(k):
        teste = lista_subsets[i]
        treino = []
        for subset in lista_subsets:
            if subset != teste:
                treino.append(subset)
        kfolds.append((teste, treino))
        
    return kfolds

In [59]:
def montar_instancias(lista_indices, data_set, treino = True):
    instancias = []
    
    if treino:
        for lista in lista_indices:
            for indice in lista:
                instancias.append(data_set[indice])
    else:
        for indice in lista_indices:
            instancias.append(data_set[indice])
            
    return np.array(instancias)

In [60]:
def GradienteDescendente(X, Y, epocas, alpha):
    Erros = []
    
    #Cria um array para 
    W = np.zeros((X.shape[1], 1))
    
    for t in range(0, epocas):
        #Calculo do erro na iteração t
        Y_ = (X @ W)
        Erro  = Y - sigmoide(Y_)
        Erros.append(np.mean(Erro))
        
        #Ajuste dos pesos na iteração t
        X_ = Erro * X
        Media = np.mean((Erro), axis = 0, keepdims=True)
        W = W + alpha * Media.T
        
    return (W, Y_, Erros)

### Função principal

In [245]:
data_set = np.genfromtxt('breastcancer.csv', delimiter = ',')

#Por questão de legibilidade
qtd_linhas, qtd_colunas = data_set.shape

data_set = np.c_[np.ones((1, qtd_linhas)).T, data_set]

#Calcula os conjuntos de teste e treino
kfolds = kfold(qtd_linhas) 

testes = []
treinos = []
for teste, treino in kfolds:
    testes.append(montar_instancias(teste, data_set,  treino=False))
    treinos.append(montar_instancias(treino, data_set, treino=True))
    
resultado_treino = []
for treino in treinos:
    X_treino = treino[:, :-1]
    Y_treino = treino[:, [-1]]
    W, Y_, Erros = GradienteDescendente(X_treino, Y_treino, 2000, 0.001)
    
    resultado_treino.append((W, Erros[-1]))
    
for i, teste in enumerate(testes):
    X_teste = testes[i][:, :-1]
    Y_teste = testes[i][:, [-1]]
    
    Y_ = X_teste @ W
    erro_teste = np.mean(( Y_teste - sigmoide(Y_)))
    
    resultado.append((W, Erros[-1], erro_teste))
    

## Analise do Discriminante Gaussiano

### Funções preliminares

In [43]:
data_set = np.genfromtxt('breastcancer.csv', delimiter = ',')

#Por questões de legibilidade
qtd_linhas, qtd_colunas = data_set.shape

#Conta a quantidade de classes, cada posição no vetor representa uma classe
N_classes = [0, 0]
for inf in data_set:
    if inf[-1] == 0.0:
        N_classes[0] += 1
    else:
        N_classes[1] += 1
        
N = qtd_linhas
# Prob_classe é um vetor onde cada posíção representa uma classe e o valor associado a probabilidade
pob_classe = [0, 0]
for c in range(0, Nk):
    pob_classe[c] = N_classes[c] / N

#Calculando Médias das classes:
medias = np.zeros(2)
for inf in data_set:
    if inf[-1] == 0.0:
        medias[0] += inf[:-1].sum()
    else:
        medias[1] += inf[:-1].sum()

medias[0] = medias[0] / N_classes[0]
medias[1] = medias[1] / N_classes[1]

#Calculando os desvios padrão de cada classe
sigmas = np.zeros(2)
for inf in data_set:
    if inf[-1] == 0:
        sigmas[0] += (inf[:-1] - medias[0]) @ (inf[:-1] - medias[0]).T
    else:
        sigmas[1] += (inf[:-1] - medias[1]) @ (inf[:-1] - medias[1]).T
        
sigmas[0] = sigmas[0] / (N_classes[0] - 1)
sigmas[1] = sigmas[1] / (N_classes[1] - 1)

### Função Principal

In [61]:
#Calcula os conjuntos de teste e treino de forma randomicamente diferente da versão anterior
kfolds = kfold(qtd_linhas, seed = 50) 

testes = []
treinos = []
for teste, treino in kfolds:
    testes.append(montar_instancias(teste, data_set,  treino=False))
    treinos.append(montar_instancias(treino, data_set, treino=True))


